# Elo contest
created by Elior Nehemya & Eden Levy

---

Elo competition focused on 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import time

from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.layers import Dense, Conv2D, Flatten, Input,Dropout,BatchNormalization, Embedding, concatenate
from keras.regularizers import l2
from keras.utils import to_categorical
from keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix
from math import sqrt

Using TensorFlow backend.


In [47]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
df_train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/train.csv')
df_test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/test.csv')

Loading the data

# Starting from the bottom!!
Creating a baseline ML prediction just from train.csv

In [49]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [0]:
X_train = df_train.iloc[:,2:]
X_train = X_train.drop(['target'], axis=1)
Y_train = df_train['target']

x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=10)


baseline models:

In [55]:
clf = Lasso(alpha=0.2)
clf.fit(x_train,y_train)

clf_pred = clf.predict(x_test)
rms = sqrt(mean_squared_error(y_test, clf_pred))
print(rms)


3.8008292845419827


In [0]:
toPred = df_test.iloc[:,2:]
pred2 = clf.predict(toPred)
sub = pd.DataFrame(data=df_test.iloc[:,1])
sub['target'] = pred2
sub.to_csv('submission4.csv', encoding='utf-8', index=False)

In [0]:
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

svr_pred_test = svr_rbf.predict(x_test)
rms = sqrt(mean_squared_error(y_test, svr_pred_test))
print(rms)



In [53]:
regr_1 = DecisionTreeRegressor(max_depth=5)
regr_1.fit(x_train, y_train)
pred = regr_1.predict(x_test)
rms = sqrt(mean_squared_error(y_test, pred))
print(rms)



3.8002729686269037


In [0]:
toPred = df_test.iloc[:,2:]
pred2 = regr_1.predict(toPred)
sub = pd.DataFrame(data=df_test.iloc[:,1])
sub['target'] = pred2
sub.to_csv('submission2.csv', encoding='utf-8', index=False)


In [0]:
model = Sequential()
  
model.add(Dense(9, activation='elu', input_shape=(3,)))
model.add(BatchNormalization())
model.add(Dense(6, activation='elu'))
model.add(Dense(1, activation='linear'))
  
model.compile(loss='mse', optimizer='adam')
  
model.summary()
history1 = model.fit(X_train,Y_train,validation_split=0.2,shuffle=True,epochs=10)


# Category Embeding
Here we will use category embeding in order to get more information from our category features


In [0]:
data = X_train[:]
F1 = {p:i for (i,p) in enumerate(data['feature_1'].unique())}
F2 = {p:i for (i,p) in enumerate(data['feature_2'].unique())}
F3 = {p:i for (i,p) in enumerate(data['feature_3'].unique())}

processed_data = data.copy()
processed_data['F1'] = [F1[x] for x in data['feature_1']]
processed_data['F2'] = [F2[x] for x in data['feature_2']]
processed_data['F3'] = [F3[x] for x in data['feature_3']]
processed_data = processed_data.loc[:,'F1':'F3']

target = Y_train[:]

F1_inp = Input(shape=(1,),dtype='int64')
F2_inp = Input(shape=(1,),dtype='int64')
F3_inp = Input(shape=(1,),dtype='int64')

F1_emb = Embedding(len(F1),4,input_length=1, embeddings_regularizer=l2(1e-6))(F1_inp)
F2_emb = Embedding(len(F2),2,input_length=1, embeddings_regularizer=l2(1e-6))(F2_inp)
F3_emb = Embedding(len(F3),2,input_length=1, embeddings_regularizer=l2(1e-6))(F3_inp)

In [0]:

x = concatenate([F1_emb,F2_emb,F3_emb], name='concat')

x = Flatten()(x)

x = Dense(64,activation='relu')(x)
x = Dense(128,activation='relu')(x)
x = Dense(64,activation='relu')(x)
x = Dense(1, activation='linear')(x)
nn_model = Model([F1_inp,F2_inp,F3_inp],x)
nn_model.compile(loss = 'mse',optimizer='adam')

In [81]:
history1 = nn_model.fit([processed_data['F1'],processed_data['F2'],processed_data['F3']],target,validation_split=0.2,epochs=10)

Train on 161533 samples, validate on 40384 samples
Epoch 1/10
161533/161533 [==============================] - 11s 66us/step - loss: 14.6920 - val_loss: 15.3615
Epoch 2/10
161533/161533 [==============================] - 10s 59us/step - loss: 14.6874 - val_loss: 15.3621
Epoch 3/10
161533/161533 [==============================] - 9s 58us/step - loss: 14.6855 - val_loss: 15.3543
Epoch 4/10
161533/161533 [==============================] - 9s 59us/step - loss: 14.6837 - val_loss: 15.3696
Epoch 5/10
161533/161533 [==============================] - 9s 59us/step - loss: 14.6847 - val_loss: 15.3573
Epoch 6/10
161533/161533 [==============================] - 9s 59us/step - loss: 14.6811 - val_loss: 15.3584
Epoch 7/10
161533/161533 [==============================] - 9s 58us/step - loss: 14.6832 - val_loss: 15.3538
Epoch 8/10
161533/161533 [==============================] - 9s 58us/step - loss: 14.6834 - val_loss: 15.3538
Epoch 9/10
161533/161533 [==============================] - 9s 58us/step - 

In [6]:
pred_data = x_test[:]
processed_pred = pred_data.copy()
processed_pred['F1'] = [F1[x] for x in pred_data['feature_1']]
processed_pred['F2'] = [F2[x] for x in pred_data['feature_2']]
processed_pred['F3'] = [F3[x] for x in pred_data['feature_3']]
processed_pred = processed_pred.loc[:,'F1':'F3']


x_test1 = [processed_pred[f] for f in processed_data.columns]

y_predicted = nn_model.predict(x_test1)
rms = np.sqrt(mean_squared_error(y_test, y_predicted))
print (rms)


3.8191095017851704


In [0]:
nn_model.save_weights('/content/gdrive/My Drive/Colab Notebooks/project_2/my_embedding_weights.h5')

now we will try add more features to our model

In [0]:
nn_model.load_weights('/content/gdrive/My Drive/Colab Notebooks/project_2/my_embedding_weights.h5')

now we will use embedding together with new data v1

In [0]:
df_train_v1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/train_merge_new_transactions.csv')
df_test_v1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/test_merge_new_transactions.csv')
X_train = df_train_v1.iloc[:,7:]
#X_train = X_train.drop(['target'], axis=1)
#X_scaled = preprocessing.scale(X_train)
#X_scaled = pd.DataFrame(data=X_scaled)
Y_train = df_train_v1['target']

In [11]:
X_train.shape

(201917, 21)

In [13]:
emb_model = Model(input = nn_model.input, output = nn_model.get_layer('concat').output)
emb_out = emb_model.predict(x_test1)
new_train = X_train

emb_out = emb_out.reshape((emb_out.shape[0]), emb_out.shape[2])
pred =  pd.DataFrame(emb_out)
new_train = pd.concat([new_train, pred], axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("co...)`
  """Entry point for launching an IPython kernel.


In [24]:
new_train.shape

(201917, 29)

In [0]:
inp = Input(shape=(29,))
z = Dense(128, activation='relu')(inp)
z = Dense(256, activation='relu')(z)
z = Dense(256, activation='relu')(z)
z = Dense(1, activation='linear')(z)

model = Model(inputs = inp, outputs= z )
model.compile(loss = 'mse', optimizer = 'adam', )

In [26]:
history = model.fit(new_train, Y_train, batch_size=64, epochs=5)


AttributeError: ignored

# Adding more features v1


working on prepared data v1 only new marchent transactions merge


In [0]:
df_train_v1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/train_merge_new_transactions.csv')
df_test_v1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/test_merge_new_transactions.csv')


In [0]:
df_train_v1.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
first_active_month,2017-06-01,2017-01-01,2016-08-01,2017-09-01,2017-11-01
card_id,C_ID_92a2005557,C_ID_3d0044924f,C_ID_d639edf6cd,C_ID_186d6a6901,C_ID_cdbd2c0db2
feature_1,5,4,2,4,1
feature_2,2,1,2,3,3
feature_3,1,0,0,0,0
target,-0.820283,0.392913,0.688056,0.142495,-0.159749
elapsed_time,245,396,549,153,92
new_transactions_count,23,6,1,7,36
new_purchase_amount_sum,-13.24,-4.355,-0.7,-4.656,-19.92


adding normalization 

In [0]:
X_train = df_train_v1.iloc[:,7:]
#X_train = X_train.drop(['target'], axis=1)
X_scaled = preprocessing.scale(X_train)
X_scaled = pd.DataFrame(data=X_scaled)
Y_train = df_train_v1.iloc[:,6]
x_train, x_test, y_train, y_test = train_test_split(X_scaled, Y_train, test_size=0.2, random_state=10)

clf = Lasso(alpha=0.2)
clf.fit(x_train,y_train)

clf_pred = clf.predict(x_test)
rms = sqrt(mean_squared_error(y_test, clf_pred))
print(rms)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


3.789670319444814


performing lasso on v1 data got 3.905 at submission

In [0]:
X_train = df_train_v1.iloc[:,7:]
#X_train = X_train.drop(['target'], axis=1)
Y_train = df_train_v1.iloc[:,6]
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=10)

clf = Lasso(alpha=0.2)
clf.fit(X_train,Y_train)

clf_pred = clf.predict(x_test)
rms = sqrt(mean_squared_error(y_test, clf_pred))
print(rms)

3.7771473173106607


In [0]:
toPred_v1 = df_test_v1.iloc[:,3:]
clf_pred = clf.predict(toPred_v1)

sub = pd.DataFrame(data=df_test_v1.iloc[:,2])
sub['target'] = clf_pred
sub.to_csv('submission4.csv', encoding='utf-8', index=False)

In [0]:
sub.shape

(123623, 2)

new attemp over v1 data now we will form nn model under scaled data

*   got 3.830 rmse nice!!!
   






In [0]:
model = Sequential()
  
model.add(Dense(50, activation='relu', input_shape=(21,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))

model.add(Dense(1, activation='linear'))
  
model.compile(loss='mse', optimizer='adam')
  
model.summary()
history1 = model.fit(X_scaled,Y_train,validation_split=0.2,shuffle=True,epochs=10)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_56 (Dense)             (None, 50)                1100      
_________________________________________________________________
dense_57 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_58 (Dense)             (None, 256)               25856     
_________________________________________________________________
dense_59 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_60 (Dense)             (None, 1)                 257       
Total params: 98,105
Trainable params: 98,105
Non-trainable params: 0
_________________________________________________________________
Train on 161533 samples, validate on 40384 samples
Epoch 1/10
161533/161533 [==============================] - 17s 106us/step - loss: 14.2

In [0]:
toPred_scalev1 = preprocessing.scale(df_test_v1.iloc[:,6:])

pred_nn_v1 = model.predict(toPred_scalev1)
sub = pd.DataFrame(data=df_test_v1.iloc[:,2])
sub['target'] = pred_nn_v1
sub.to_csv('submission5.csv', encoding='utf-8', index=False)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


adding some modifications to nn model


In [0]:
model = Sequential()
  
model.add(Dense(128, activation='relu', input_shape=(21,)))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='linear'))
  
model.compile(loss='mse', optimizer='adam')
  
model.summary()
history1 = model.fit(X_scaled,Y_train,validation_split=0.2,shuffle=True,epochs=20)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_66 (Dense)             (None, 128)               2816      
_________________________________________________________________
batch_normalization_10 (Batc (None, 128)               512       
_________________________________________________________________
dense_67 (Dense)             (None, 256)               33024     
_________________________________________________________________
batch_normalization_11 (Batc (None, 256)               1024      
_________________________________________________________________
dense_68 (Dense)             (None, 256)               65792     
_________________________________________________________________
batch_normalization_12 (Batc (None, 256)               1024      
_________________________________________________________________
dense_69 (Dense)             (None, 256)               65792     
__________

#  Adding more features v2
---
V2 data created by merging train/test with historical_transactions data




In [0]:
df_train_v2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/train_merge_history_transactions.csv')
df_test_v2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/test_merge_history_transactions.csv')

In [0]:
df_train_v2.head()

,Unnamed: 0,first_active_month,card_id,feature_1,feature_2,feature_3,target,elapsed_time,hist_transactions_count,hist_authorized_flag_sum,...,hist_installments_std,hist_purchase_month_mean,hist_purchase_month_max,hist_purchase_month_min,hist_purchase_month_std,hist_purchase_date_ptp,hist_purchase_date_max,hist_purchase_date_min,hist_month_lag_min,hist_month_lag_max
0,0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,245,260,247.0,...,0.123314,8.057692,12,1,3.474193,20977987.0,1.519551e+09,1.498573e+09,-8,0
1,1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,396,350,339.0,...,1.510777,6.220000,12,1,3.848142,33717687.0,1.517438e+09,1.483720e+09,-12,0
2,2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,549,43,41.0,...,0.000000,4.558140,12,1,3.275467,35635623.0,1.519759e+09,1.484123e+09,-13,0
3,3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,153,77,77.0,...,0.588974,7.740260,12,1,3.904797,13375339.0,1.519818e+09,1.506443e+09,-5,0
4,4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,92,133,128.0,...,1.896862,5.406015,12,1,5.003086,9405641.0,1.519850e+09,1.510445e+09,-3,0


In [0]:
X_train = df_train_v2.iloc[:,7:]
#X_train = X_train.drop(['target'], axis=1)
X_scaled = preprocessing.scale(X_train)
X_scaled = pd.DataFrame(data=X_scaled)
Y_train = df_train_v2.iloc[:,6]
x_train, x_test, y_train, y_test = train_test_split(X_scaled, Y_train, test_size=0.2, random_state=10)


#get some ML banchmark to comper 
clf = Lasso(alpha=0.2)
clf.fit(x_train,y_train)

clf_pred = clf.predict(x_test)
rms = sqrt(mean_squared_error(y_test, clf_pred))
print(rms)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


3.7912113873820426


In [0]:
X_scaled.describe()

performing nn model over v2 data

In [0]:
model = Sequential()
  
model.add(Dense(128, activation='relu', input_shape=(23,)))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='linear'))
  
model.compile(loss='mse', optimizer='adam')
  
model.summary()
history1 = model.fit(X_scaled,Y_train,validation_split=0.2,shuffle=True,epochs=20)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               3072      
_________________________________________________________________
batch_normalization_5 (Batch (None, 128)               512       
_________________________________________________________________
dense_7 (Dense)              (None, 256)               33024     
_________________________________________________________________
batch_normalization_6 (Batch (None, 256)               1024      
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
batch_normalization_7 (Batch (None, 256)               1024      
_________________________________________________________________
dense_9 (Dense)              (None, 256)               65792     
__________

In [0]:
toPred_scale_v2 = preprocessing.scale(df_test_v2.iloc[:,6:])

pred_nn_v2 = model.predict(toPred_scale_v2)
sub = pd.DataFrame(data=df_test_v1.iloc[:,2])
sub['target'] = pred_nn_v1
sub.to_csv('submission5.csv', encoding='utf-8', index=False)

less deeper nn model

*   got 3.824 !! some improvment from v1




In [0]:
model = Sequential()
  
model.add(Dense(50, activation='relu', input_shape=(23,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='linear'))
  
model.compile(loss='mse', optimizer='adam')
  
model.summary()
history1 = model.fit(X_scaled,Y_train,validation_split=0.2,shuffle=True,epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_47 (Dense)             (None, 50)                1200      
_________________________________________________________________
dense_48 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_49 (Dense)             (None, 256)               25856     
_________________________________________________________________
dense_50 (Dense)             (None, 256)               65792     
_________________________________________________________________
batch_normalization_10 (Batc (None, 256)               1024      
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 257       
Total params: 99,229
Trainable params: 98,717
Non-trainable params: 512
_________________________________________________________________
Trai

In [0]:
toPred_scale_v2 = preprocessing.scale(df_test_v2.iloc[:,6:])

pred_nn_v2 = model.predict(toPred_scale_v2)
sub = pd.DataFrame(data=df_test_v2.iloc[:,2])
sub['target'] = pred_nn_v2
sub.to_csv('submission10.csv', encoding='utf-8', index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


simplier nn model

In [0]:
model = Sequential()
  
model.add(Dense(50, activation='relu', input_shape=(23,)))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='linear'))
  
model.compile(loss='mse', optimizer='adamax')
  
model.summary()
history1 = model.fit(X_scaled,Y_train,validation_split=0.2,shuffle=True,epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 50)                1200      
_________________________________________________________________
dense_45 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_46 (Dense)             (None, 1)                 26        
Total params: 2,501
Trainable params: 2,501
Non-trainable params: 0
_________________________________________________________________
Train on 161533 samples, validate on 40384 samples
Epoch 1/10
161533/161533 [==============================] - 9s 53us/step - loss: 14.3364 - val_loss: 14.6738
Epoch 2/10
161533/161533 [==============================] - 7s 45us/step - loss: 14.1457 - val_loss: 14.6397
Epoch 3/10
161533/161533 [==============================] - 7s 45us/step - loss: 14.0894 - val_loss: 14.6192
Epoch 4/10
161533/161533 [

# Adding more features v3

---

V3 is combination of train/test & historical & new transactions

In [0]:
df_train_v3 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/train_merge_new&H_transactions.csv')
df_test_v3 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/test_merge_new&H_transactions.csv')

In [0]:
df_train_v3.head()

In [0]:
X_train = df_train_v3.iloc[:,7:]
#X_train = X_train.drop(['target'], axis=1)
X_scaled = preprocessing.scale(X_train)
X_scaled = pd.DataFrame(data=X_scaled)
Y_train = df_train_v3.iloc[:,6]
x_train, x_test, y_train, y_test = train_test_split(X_scaled, Y_train, test_size=0.2, random_state=10)


#get some ML banchmark to comper 
clf = Lasso(alpha=0.2)
clf.fit(x_train,y_train)

clf_pred = clf.predict(x_test)
rms = sqrt(mean_squared_error(y_test, clf_pred))
print(rms)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


3.7856005548649665


In [0]:
model = Sequential()
  
model.add(Dense(50, activation='relu', input_shape=(48,)))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='linear'))
  
model.compile(loss='mse', optimizer='adam')
  
model.summary()
history1 = model.fit(X_scaled,Y_train,validation_split=0.2,shuffle=True,epochs=20)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_86 (Dense)             (None, 50)                2450      
_________________________________________________________________
dense_87 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_88 (Dense)             (None, 1)                 26        
Total params: 3,751
Trainable params: 3,751
Non-trainable params: 0
_________________________________________________________________
Train on 161533 samples, validate on 40384 samples
Epoch 1/20
161533/161533 [==============================] - 12s 74us/step - loss: 14.0332 - val_loss: 14.2389
Epoch 2/20
161533/161533 [==============================] - 9s 58us/step - loss: 13.8496 - val_loss: 14.0805
Epoch 3/20
161533/161533 [==============================] - 9s 59us/step - loss: 13.6830 - val_loss: 14.0906
Epoch 4/20
161533/161533 

In [0]:
model = Sequential()
  
model.add(Dense(50, activation='relu', input_shape=(45,)))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='linear'))
  
model.compile(loss='mse', optimizer='adam')
  
model.summary()
history1 = model.fit(X_scaled,Y_train,validation_split=0.2,shuffle=True,epochs=20)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_75 (Dense)             (None, 50)                2300      
_________________________________________________________________
dense_76 (Dense)             (None, 100)               5100      
_________________________________________________________________
batch_normalization_15 (Batc (None, 100)               400       
_________________________________________________________________
dense_77 (Dense)             (None, 256)               25856     
_________________________________________________________________
dense_78 (Dense)             (None, 256)               65792     
_________________________________________________________________
batch_normalization_16 (Batc (None, 256)               1024      
_________________________________________________________________
dense_79 (Dense)             (None, 1)                 257       
Total para

nn deep model 

*   got 3.787!!!




In [0]:
model = Sequential()
  
model.add(Dense(128, activation='relu', input_shape=(48,)))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='linear'))
  
model.compile(loss='mse', optimizer='adam')
  
model.summary()
history1 = model.fit(X_scaled,Y_train,validation_split=0.2,shuffle=True,epochs=20)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_94 (Dense)             (None, 128)               6272      
_________________________________________________________________
batch_normalization_21 (Batc (None, 128)               512       
_________________________________________________________________
dense_95 (Dense)             (None, 256)               33024     
_________________________________________________________________
batch_normalization_22 (Batc (None, 256)               1024      
_________________________________________________________________
dense_96 (Dense)             (None, 256)               65792     
_________________________________________________________________
batch_normalization_23 (Batc (None, 256)               1024      
_________________________________________________________________
dense_97 (Dense)             (None, 256)               65792     
__________

In [0]:
toPred_scale_v3 = preprocessing.scale(df_test_v3.iloc[:,6:])

pred_nn_v3 = model.predict(toPred_scale_v3)
sub = pd.DataFrame(data=df_test_v3.iloc[:,2])
sub['target'] = pred_nn_v3
sub.to_csv('submission11.csv', encoding='utf-8', index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


# Adding more features V4

---

now we add one hot encoding for the categorial fetures, based on hist+new transactions 

In [0]:
df_train_v4 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/V4_train_new+hist+main.csv')
df_test_v4 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/V4_test_new+hist+main.csv')
df_train_v4 = df_train_v4.fillna(0)
df_test_v4 = df_test_v4.fillna(0)

In [102]:
X_train = df_train_v4.iloc[:,2:]
X_train = X_train.drop(['target'], axis=1)
X_train = X_train.drop(['first_active_month'], axis=1)
#X_scaled = preprocessing.scale(X_train)
#X_scaled = pd.DataFrame(data=X_scaled)
Y_train = df_train_v4.loc[:,'target']
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=10)


#get some ML banchmark to comper 
clf = Lasso(alpha=0.2)
clf.fit(x_train,y_train)

clf_pred = clf.predict(x_test)
rms = sqrt(mean_squared_error(y_test, clf_pred))
print(rms)

ValueError: ignored

testing our deep nn model over v4 data

In [0]:
df_test_v4.head(3).T

,0,1,2
Unnamed: 0,0,1,2
card_id,C_ID_0ab67a22ab,C_ID_130fd0cbdd,C_ID_b709037bc5
elapsed_time,306,396,184
first_active_month,2017-04-01,2017-01-01,2017-08-01
hist_authorized_flag_mean,0.647059,0.987179,0.692308
hist_authorized_flag_sum,44,77,9
hist_category_1_mean,0.338235,0.025641,0.0769231
hist_category_1_sum,23,2,1
hist_category_2_1_mean,0.661765,0,0.0769231
hist_category_2_1_sum,45,0,1


deep nn model over v4 data
*   got 3.778




In [45]:
model = Sequential()
  
model.add(Dense(128, activation='relu', input_shape=(91,)))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='linear'))
  
model.compile(loss='mse', optimizer='adam')
  
model.summary()
history1 = model.fit(X_scaled,Y_train,validation_split=0.2,shuffle=True,epochs=10)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               11776     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
batch_normalization_3 (Batch (None, 256)               1024      
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
__________

In [0]:
Xpred=df_test_v4.iloc[:,2:]
Xpred.head(3)


,elapsed_time,first_active_month,hist_authorized_flag_mean,hist_authorized_flag_sum,hist_category_1_mean,hist_category_1_sum,hist_category_2_1_mean,hist_category_2_1_sum,hist_category_2_2_mean,hist_category_2_2_sum,...,feature_1_0,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_2_0,feature_2_1,feature_2_2,feature_3_0,feature_3_1
0,306.0,2017-04-01,0.647059,44.0,0.338235,23.0,0.661765,45.0,0.0,0.0,...,0,0,1,0,0,0,0,1,0,1
1,396.0,2017-01-01,0.987179,77.0,0.025641,2.0,0.000000,0.0,0.0,0.0,...,0,1,0,0,0,0,0,1,1,0
2,184.0,2017-08-01,0.692308,9.0,0.076923,1.0,0.076923,1.0,0.0,0.0,...,0,0,0,0,1,1,0,0,0,1


In [0]:
Xpred=df_test_v4.iloc[:,2:]
Xpred = Xpred.drop(['first_active_month'], axis=1)
Xpred = Xpred.drop(['target'], axis=1)
toPred_scale_v4 = preprocessing.scale(Xpred)

pred_nn_v4 = model.predict(toPred_scale_v4)
sub = pd.DataFrame(data=df_test_v4.iloc[:,1])
sub['target'] = pred_nn_v4
sub.to_csv('submission12.csv', encoding='utf-8', index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [0]:
model = Sequential()
  
model.add(Dense(91, activation='relu', input_shape=(91,)))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='linear'))
  
model.compile(loss='mse', optimizer='adam')
  
model.summary()
history1 = model.fit(X_scaled,Y_train,validation_split=0.2,shuffle=True,epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 91)                8372      
_________________________________________________________________
batch_normalization_15 (Batc (None, 91)                364       
_________________________________________________________________
dense_24 (Dense)             (None, 256)               23552     
_________________________________________________________________
dense_25 (Dense)             (None, 256)               65792     
_________________________________________________________________
batch_normalization_16 (Batc (None, 256)               1024      
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 257       
Total params: 99,361
Trainable params: 98,667
Non-trainable params: 694
_________________________________________________________________
Trai

In [0]:
model = Sequential()
  
model.add(Dense(50, activation='relu', input_shape=(91,)))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='linear'))
  
model.compile(loss='mse', optimizer='adam')
  
model.summary()
history1 = model.fit(X_scaled,Y_train,validation_split=0.2,shuffle=True,epochs=20)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 50)                4600      
_________________________________________________________________
dense_12 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 26        
Total params: 5,901
Trainable params: 5,901
Non-trainable params: 0
_________________________________________________________________
Train on 161533 samples, validate on 40384 samples
Epoch 1/20
161533/161533 [==============================] - 8s 51us/step - loss: 14.0995 - val_loss: 14.3165
Epoch 2/20
161533/161533 [==============================] - 7s 46us/step - loss: 13.7305 - val_loss: 14.1784
Epoch 3/20
161533/161533 [==============================] - 7s 46us/step - loss: 13.6048 - val_loss: 14.2221
Epoch 4/20
161533/161533 [

another try now with lgb model


In [11]:
xtrain.head(3).T


,0,1,2
Unnamed: 0,0,1,2
elapsed_time,245,396,549
first_active_month,2017-06-01,2017-01-01,2016-08-01
hist_authorized_flag_mean,0.95,0.968571,0.953488
hist_authorized_flag_sum,247,339,41
hist_category_1_mean,0,0.0885714,0
hist_category_1_sum,0,31,0
hist_category_2_1_mean,0.988462,0.911429,0.0930233
hist_category_2_1_sum,257,319,4
hist_category_2_2_mean,0,0,0


In [0]:
xtrain = df_train_v4
xtrain = xtrain.drop('first_active_month', axis=1)
ytrain = df_train_v4['target']
xtrain = xtrain.drop('target',axis=1)
xtest = df_test_v4
# prepare for modeling
id_train = xtrain['card_id'].copy(); xtrain.drop('card_id', axis = 1, inplace = True)
id_test = xtest['card_id'].copy(); xtest.drop('card_id', axis = 1, inplace = True)


nfolds = 10
folds = KFold(n_splits= nfolds, shuffle=True, random_state=15)

In [0]:
xtrain = xtrain.iloc[:,1:]
xtest = xtest.drop('target',axis=1)
xtest = xtest.drop('first_active_month',axis=1)
xtest = xtest.iloc[:,1:]
xtrain = xtrain.fillna(0)
xtest = xtest.fillna(0)

In [0]:
param = {'num_leaves': 50,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': 10,
         'learning_rate': 0.005,
         "min_child_samples": 100,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1}

In [39]:
feature_importance_df = np.zeros((xtrain.shape[1], nfolds))
mvalid = np.zeros(len(xtrain))
mfull = np.zeros(len(xtest))


start = time.time()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(xtrain.values, ytrain.values)):
    print('----')
    print("fold n°{}".format(fold_))
    
    x0,y0 = xtrain.iloc[trn_idx], ytrain[trn_idx]
    x1,y1 = xtrain.iloc[val_idx], ytrain[val_idx]
    
    trn_data = lgb.Dataset(x0, label= y0); val_data = lgb.Dataset(x1, label= y1)
    
    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], 
                    verbose_eval=500, early_stopping_rounds = 150)
    mvalid[val_idx] = clf.predict(x1, num_iteration=clf.best_iteration)
    
    feature_importance_df[:, fold_] = clf.feature_importance()
    
    mfull += clf.predict(xtest, num_iteration=clf.best_iteration) / folds.n_splits

----
fold n°0
Training until validation scores don't improve for 150 rounds.
[500]	training's rmse: 3.56105	valid_1's rmse: 3.74187
[1000]	training's rmse: 3.47997	valid_1's rmse: 3.72519
[1500]	training's rmse: 3.42199	valid_1's rmse: 3.72132
[2000]	training's rmse: 3.37389	valid_1's rmse: 3.72049
Early stopping, best iteration is:
[1880]	training's rmse: 3.38491	valid_1's rmse: 3.72024
----
fold n°1
Training until validation scores don't improve for 150 rounds.
[500]	training's rmse: 3.56997	valid_1's rmse: 3.67716
[1000]	training's rmse: 3.48891	valid_1's rmse: 3.66438
[1500]	training's rmse: 3.4294	valid_1's rmse: 3.66114
Early stopping, best iteration is:
[1527]	training's rmse: 3.42648	valid_1's rmse: 3.66107
----
fold n°2
Training until validation scores don't improve for 150 rounds.
[500]	training's rmse: 3.57331	valid_1's rmse: 3.61555
[1000]	training's rmse: 3.49169	valid_1's rmse: 3.60058
[1500]	training's rmse: 3.43217	valid_1's rmse: 3.59664
[2000]	training's rmse: 3.38366

lgbm got 3.722!!

In [0]:
xsub = pd.DataFrame()
xsub['card_id']  = id_test
xsub['target'] = mfull
xsub.to_csv('submission14.csv', encoding='utf-8', index=False)

# Outlier clasification

---

here we try new direction to classify outlier and than use different regression for each cluster norm/outlier


first we need to label the data: 1-outlier 0-normal

In [0]:
df_train_v3 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/train_merge_new&H_transactions.csv')
df_test_v3 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/project_2/test_merge_new&H_transactions.csv')

X_train = df_train_v3.iloc[:,7:]

X_train['outliers_target'] = 0
X_train.loc[df_train_v3['target'] < -30, 'outliers_target'] = 1
X_train['outliers_target'].value_counts()

outliers = X_train.loc[X_train['outliers_target'] == 1]
norm = X_train.loc[X_train['outliers_target'] == 0]

balanced = norm.sample(n=10000)
balanced = balanced.append(outliers)


In [0]:
balanced.head(3)

,elapsed_time,hist_transactions_count,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,...,new_installments_std,new_purchase_month_mean,new_purchase_month_max,new_purchase_month_min,new_purchase_month_std,new_purchase_date_ptp,new_purchase_date_max,new_purchase_date_min,new_month_lag_min,new_month_lag_max
67468,915,32,21.0,0.656250,-18.102047,-0.565689,-0.027438,-0.733429,0.163252,59,...,0.0,3.0,3.0,3.0,0.000000,0.0,1.521044e+09,1.521044e+09,1.0,1.0
40792,306,19,18.0,0.947368,-11.924168,-0.627588,-0.346136,-0.732933,0.152228,0,...,0.0,2.0,2.0,2.0,0.000000,1021998.0,1.518890e+09,1.517868e+09,1.0,1.0
138750,854,52,38.0,0.730769,-34.827023,-0.669750,-0.313708,-0.745405,0.103801,9,...,0.0,3.2,4.0,3.0,0.447214,2981123.0,1.523427e+09,1.520446e+09,1.0,2.0


In [0]:
#X_train = X_train.drop(['target'], axis=1)
Y_train = balanced['outliers_target']
balanced = balanced.drop(['outliers_target'], axis=1)
X_scaled = preprocessing.scale(balanced)
X_scaled = pd.DataFrame(data=X_scaled)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, Y_train, test_size=0.2, random_state=10)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [0]:
model = Sequential()
  
model.add(Dense(50, activation='relu', input_shape=(45,)))
model.add(BatchNormalization())

model.add(Dense(50, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='softmax'))
  
model.compile(loss='binary_crossentropy', optimizer='adam')
  
model.summary()
history1 = model.fit(X_scaled,Y_train,shuffle=True,epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 50)                2300      
_________________________________________________________________
batch_normalization_7 (Batch (None, 50)                200       
_________________________________________________________________
dense_11 (Dense)             (None, 50)                2550      
_________________________________________________________________
batch_normalization_8 (Batch (None, 50)                200       
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 51        
Total params: 5,301
Trainable params: 5,101
Non-trainable params: 200
_________________________________________________________________
Epoch 1/10
12207/12207 [==============================] - 2s 134us/step - loss: 13.0600
Epoch 2/10
12207/12207 [==========================

In [0]:
clf = RandomForestClassifier(n_estimators=100, max_depth=50, random_state=0)
clf.fit(X_scaled, Y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [0]:
X_train.head()

,elapsed_time,hist_transactions_count,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,...,new_purchase_month_mean,new_purchase_month_max,new_purchase_month_min,new_purchase_month_std,new_purchase_date_ptp,new_purchase_date_max,new_purchase_date_min,new_month_lag_min,new_month_lag_max,outliers_target
0,245,260,247.0,0.950000,-165.968730,-0.638341,2.258395,-0.739395,0.212139,4,...,3.478261,4.0,3.0,0.510754,4742309.0,1.525001e+09,1.520259e+09,1.0,2.0,0
1,396,350,339.0,0.968571,-210.006330,-0.600018,4.630300,-0.742400,0.384967,543,...,2.500000,3.0,2.0,0.547723,4887632.0,1.522393e+09,1.517505e+09,1.0,2.0,0
2,549,43,41.0,0.953488,-29.167390,-0.678311,-0.145847,-0.730138,0.087380,0,...,4.000000,4.0,4.0,0.000000,0.0,1.524937e+09,1.524937e+09,2.0,2.0,0
3,153,77,77.0,1.000000,-49.491364,-0.642745,1.445596,-0.740897,0.261624,84,...,3.714286,4.0,3.0,0.487950,3625505.0,1.524049e+09,1.520424e+09,1.0,2.0,0
4,92,133,128.0,0.962406,-48.687656,-0.366073,7.193041,-0.746156,1.352094,182,...,3.555556,4.0,3.0,0.503953,4949682.0,1.524941e+09,1.519992e+09,1.0,2.0,0


In [0]:
Y_train = X_train['outliers_target'] 
X_train1 = X_train.drop('outliers_target', axis=1)
X_train1 = preprocessing.scale(X_train1)
clf.score(X_train1,Y_train)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


0.9321503390006785

In [0]:
pred = clf.predict(X_train1)
confusion_matrix(Y_train, pred)

array([[186712,  12998],
       [   702,   1505]])

as we can see our clasifier may have high accuracy but its not enough. the confusion matrix shows that its fals positive may cost too much for the total loss.